# GHOST (General Holistic Organism Scripting Tool)

# Goals and Vision

The design intention of Ghost (the “General Holistic Organism Scripting Tool”) is to allow human authors to script behaviors for artificial characters.   Significant flexibility is desired, e.g. to support
* Purely textual chatbots, animated characters, or physical robots
* Characters controlled precisely by human-authored rules; or characters with a great deal of autonomy, where human-authored rules serve mainly to tweak parameters and indicate propensities

Ghost is envisioned as having both a textual and a graphical user interface.   The textual interface is aimed to have roughly the same level of complexity as ChatScript (on which it is heavily based), meaning that it should be usable by non-programmers who are able to deal with a fairly intricate formal syntax. 

The GUI is not yet designed at time of writing, but is intended to make Ghost authoring feasible for a broader class of content authors, and to make it easier and more rapid for everyone.   Some brief notes  regarding the envisioned GUI are given at the end of this document.

# High Level Design

Ghost is implemented as a DSL (Domain-Specific Language) within the Scheme shell associated with the OpenCog engine.    Ghost syntax closely resembles (and in most respects is identical to) ChatScript; but the Ghost interpreter is written in Scheme and runs in the OpenCog Scheme shell.   Ghost rules are interpreted into Atoms living in OpenCog’s Atomspace semantic knowledge store, and are executed within OpenCog using the OpenCog Pattern Matcher and Action Selector and associated mechanisms.

A Ghost rule has two required parts, a pattern and an action; and an optional third part, a goal or set of goals.   The general semantic is: When the pattern is observed, if the goal is being pursued by the agent, then consider taking the action. If no goal is specified, a generic goal such as “interact with others” is implicitly assumed. A Ghost “script” consists of a set of rules, which may be grouped into files called “topics.”   A topic may have some annotations associated with it, which apply to all the rules in the topic (unless overridden by annotations specific to certain rules.)

Both patterns and actions may optionally reference Scheme functions.  (Support for functions written in other languages may be added in a later version.)   An envisioned usage pattern is: Writing Ghost rules and, in coordination with the Ghost rule authoring process, adding new Scheme functions to help the rules do their things.

An example simple Ghost rule is:
```
#goal: (novelty=1)
s: ( Hi there ^no_people_around ) Who said that? ^look_around
```
This rule is somewhat trivial as it lacks variables.   But it shows the basic format; here we have:
 * Pattern = someone says “Hi there” to the agent, and there are no people around (i.e. the Scheme function no_people_around outputs True)
 * Action = the agent says “Who said that?” and then invokes the Scheme function look_around
 * Goal = novelty … so the agent should use this rule when the “novelty” goal is important to it
 
Behind the scenes, the Ghost interpreter uses OpenCog’s Pattern Matcher to match the patterns of its rules to content in OpenCog’s Atomspace knowledge store.  The specifics of this matching are simple now, but are highly customizable and can be made more sophisticated in future versions.



# Installation
**Opencog**

Ghost is a module inside [Opencog](https://github.com/opencog/opencog). So install Opencog first.

**Relex**

[RelEx](https://github.com/opencog/relex) is a dependency parser for the English language. It extracts dependency relations from Link Grammar, and adds some shallow semantic analysis. The primary use of RelEx is as a language input front-end to the OpenCog artificial general intelligence system. Ghost uses relex.

# Hello in Ghost
Open guile shell and import the necessary modules:

In [1]:
(use-modules (opencog)
             (opencog nlp)
             (opencog nlp relex2logic)
             (opencog openpsi)
             (opencog ghost)
             (opencog ghost procedures))

Next start the [relex](https://github.com/opencog/relex) server:

**Without Docker:**
```
$ cd /path/to/relex/project/directory/
$ ./opencog-server.sh
```

**With Docker:**   
When using docker, inside the started guile shell set the host for the container running relex. This way
the relex server will be looked for at the specified IP address instead of the default 127.0.0.1.

In [2]:
(set-relex-server-host)

172.21.0.2

Parse your rules using `ghost-parse` for single rule or `ghost-parse-file` for files containing rules:

In [3]:
(ghost-parse "u: (hello) hello there! ^keep")

Test the parsed rules using `test-ghost`.
When the input doesn't match any pattern:

In [4]:
(test-ghost "hi")

()

When the input matches the pattern of the previously parsed rule:

In [5]:
(test-ghost "hello")

((WordNode "hello")
 (WordNode "there")
 (WordNode "!")
)

The result sentence is returned as a list of WordNodes containing the words. To extract the sentence do the following.

In [7]:
(map cog-name (test-ghost "hello"))

(hello there !)

# Design Overview

* A GHOST rule is essentially an OpenPsi rule (aka psi-rule), which is an ImplicationLink that can be expressed as
```
context AND action -> goal
```

* That is if a pattern or context is matched action is then executed and the goal is said to be achieved.

* A goal also has an ugre to express the level of need to achieve that goal.

* Ghost rules are organized into topics. Before parsing any rules a topic can be set and all subsequent rules created will belong to that topic until another topic is set or the end of the file is reached. If no topic is set a default topic is used.

* When a GHOST rule is being created, it will firstly be passed to a parser for syntax checking and preliminary interpretation. Any rules that is not syntactically correct or with unsupported features will be rejected at this stage.

* The parser will then pass the intermediate interpretations (aka terms) to a translator that converts them into OpenCog atoms to be stored in the AtomSpace.

* Ghost action selector is responsible for selecting a rule that is applicable to a given context. When a textual input is received, rules that satisfy the given context will first be selected as candidates. A full context evaluation will then be done for each of the candidates. Action selector will pick one of them based on their satisfiability and their truth value. Satisfiability of rules are compared based on the following criteria:
   
   * Wether a pattern is matched or not
   * The strength of the rule's goal
   * The urge of the goal
   * Importance of the rule (rules just selected are considered less important)
   * Whether a rule is in the current topic (priority is given)

For example consider two rules with the same pattern but different goal strength:

In [14]:
(ghost-parse "#goal: (goal1=0.5) u: (hi) hi there ^keep")

In [15]:
(ghost-parse "#goal: (goal1=0.7) u: (hi) hello there ^keep")

In [20]:
(test-ghost "hi")

((WordNode "hello")
 (WordNode "there")
)

The action selector picks the rule with the higher goal strength.

* Once a rule is selected it will not be considered again for subsequent inputs unless it is specified otherwise via
`^keep` function or `keep` topic feature.

# Syntax

The syntax of GHOST rules is modeled heavily on [ChatScript](https://github.com/bwilcox-1234/ChatScript/blob/master/WIKI/ChatScript-Basic-User-Manual.md#rules). However, GHOST uses several ChatScript features for different purposes than they are normally used in ChatScript; and also contains some additional features.

## Topic
Rules are bundled into topics. All rules after a topic definition will belong to that rule. The topic declares its name, its keywords, and then its rules. It ends with the end of the file or a new topic declaration.

```
topic: ~NAME features [list of keywords]
```

In [33]:
(ghost-parse "topic: ~GREETING [hi hello]")

Now all rules created will be under GREETING topic. If `keep` topic feature is used all rules in the topic can be considered repeatedly by the action selector.

## Label
A rule can optionally be given a label by which it can be referred by other rules or from the guile shell:

In [35]:
(ghost-parse "u: lbl (ghost) Ghost is a behavior scripting tool ^keep")

In [36]:
(ghost-get-rule "lbl")

(ImplicationLink (stv 0.9 0.9)
   (AndLink
      (TrueLink
         (ExecutionOutputLink
            (GroundedSchemaNode "scm: ghost-execute-action")
            (ListLink
               (WordNode "Ghost")
               (WordNode "is")
               (WordNode "a")
               (WordNode "behavior")
               (WordNode "scripting")
               (WordNode "tool")
            )
         )
         (PutLink
            (StateLink
               (AnchorNode "GHOST Last Executed")
               (VariableNode "$x")
            )
            (ConceptNode "lbl")
         )
         (ExecutionOutputLink
            (GroundedSchemaNode "scm: ghost-record-executed-rule")
            (ListLink
               (ConceptNode "lbl")
            )
         )
         (PutLink
            (StateLink
               (AnchorNode "GHOST Current Topic")
               (VariableNode "$x")
            )
            (ConceptNode "GHOST GREETING")
         )
      )
      (SatisfactionLink
         (Va

## Goal
In a ghost rule the satisfiability of a context followed by execution of the action implies achievement of the given goal. Goal has value (0-1) which indicates the strength of the implication link in the rule. The higher the goal value the more likely that the execution of the action achieves that goal given that the context is satisfied.

```
context AND action ==> goal
```

There are two ways of creating goals,

**1. Top level goal(s)**

In [37]:
(ghost-parse "goal: (please_user=0.8)")

In this case, all the rules created after it will be having the same goal and the same weight, until another top level goal or the end of file is reached. For example the following rule uses the above top level goal.

In [38]:
(ghost-parse "u: lbl1 (hello) Hello sweet wonderful human")

`ghost-rule-tv` returns the truth value of the implication link as `stv mean confidence`. The mean value is the strength of the goal.

In [40]:
(ghost-rule-tv "lbl1")

(stv 0.800000 0.900000)

It is also possible to create a list of rules that are ordered.

In [43]:
(ghost-parse "ordered-goal: (please_user=0.8)")

The rules being created under ordered-goals will have a different weight, based on the order of creation. The relationship between the order and the weight forms a geometric sequence with a factor of 0.5.

For example, if there are five rules under the above please_user=0.8 goal, the first rule will have a weight of 0.4, the second one will have 0.2, the third one will have 0.1, and so on. The sum of the weights will get closer to the weight of the top level goal (0.8) if more rules are created under it.

**2. Rule level goal(s)**
In this case, the goals will only be linked to the rule created immediately after it. Top level goals will also be linked to the rule if there are any. A top level goal will be overwritten by a rule level goal if the same goal is defined. Any number of rule level goals can be specified inside the goal declaration parenthesis.

In [54]:
(ghost-parse "#goal: (novelty=0.67 please_user=0.4) u: (what be you name) I forgot")

In [55]:
(map cog-name (test-ghost "what is your name"))

(I forgot)

## Urge
A goal can have an urge to express the level of urgency to achive that goal. Tweaking the urge value of a goal affects the chances of corresponding rules to be selected by the action selector. If the urge of a goal is increased corresponding rules will be more likely to be selected and vise versa. 

The urge of a goal is 1 (maximum) by default. The default urge can be changed, and it should be done before creating the goal, for example:

In [56]:
(ghost-parse "urge: (tease_user=1 creativity=0.5)")

The urge value of the goal can be changed via OpenPsi function `psi-increase-urge GOAL VALUE` and `psi-decrease-urge GOAL VALUE`

## Lemma

## Phrase

## Concept

## Choice

## Optional

## Indefinite Wildcard

## Precise Wildcard

## Range-restricted Wildcard

## Variable

## User Variable

## Sentence Boundary

## Negation

## Function

## Unordered Matching

## Rejoinder